# Recommender System - Neural Network

The recommender here use the RMSE loss function.

## 1. Prerequisites

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings
import scipy
import scipy.io
import scipy.sparse as sp
import keras

warnings.filterwarnings('ignore')
%matplotlib inline


from sklearn.preprocessing import LabelEncoder
from keras import layers
from keras import models
from keras import optimizers

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [2]:
from helpersNeuralNet import load_data
from sklearn.model_selection import train_test_split

DATA_TRAIN_PATH = "data/data_train.csv"
ratings = load_data(DATA_TRAIN_PATH)

DATA_TEST_PATH = "data/sampleSubmission.csv"
samples = load_data(DATA_TEST_PATH)

n_users = len(ratings.user_id.unique())
n_items = len(ratings.movie_id.unique())

train, test = train_test_split(ratings, test_size=0.1, random_state=42)

In [3]:
samples.head()

,user_id,movie_id,rating
0,37,1,3
1,73,1,3
2,156,1,3
3,160,1,3
4,248,1,3


## 2. Data preprocessing

In [4]:
from sklearn import model_selection

train_x, test_x = model_selection.train_test_split(ratings, test_size=0.2, random_state=1)

In [5]:
categorical_train_y = np.zeros([train_x.shape[0], 5])
categorical_train_y[np.arange(train_x.shape[0]), train_x.rating - 1] = 1
categorical_train_y.shape

categorical_test_y = np.zeros([test_x.shape[0], 5])
categorical_test_y[np.arange(test_x.shape[0]), test_x.rating - 1] = 1
categorical_test_y.shape

(235391, 5)

## 3. Neural network

In [17]:
from keras.optimizers import Adam
from keras.regularizers import l2
from keras_radam import RAdam

def shallow_net():
    features = 48
    
    input_i = layers.Input(shape=[1])
    i = layers.Embedding(n_items + 1, features)(input_i)
    i = layers.Flatten()(i)
    i = layers.normalization.BatchNormalization()(i)

    input_u = layers.Input(shape=[1])
    u = layers.Embedding(n_users + 1, features)(input_u)
    u = layers.Flatten()(u)
    u = layers.normalization.BatchNormalization()(u)
    
    nn = layers.concatenate([i, u])
    
    nn = layers.Dense(512, activation='relu')(nn)
    nn = layers.Dropout(0.5)(nn)
    nn = layers.normalization.BatchNormalization()(nn)
    
    nn = layers.Dense(128, activation='relu')(nn)
    
    output = layers.Dense(5, activation='softmax')(nn)
    
    model = models.Model([input_i, input_u], output)
    model.compile(optimizer='adamax', loss='categorical_crossentropy')
    return model

# Optimizer : 'adamax', 'sgd', 'adam', RAdam()

In [32]:
def dense_net():
    features = 256
    
    input_i = layers.Input(shape=[1])
    i = layers.Embedding(n_items + 1, features)(input_i)
    i = layers.Flatten()(i)
    i = layers.normalization.BatchNormalization()(i)

    input_u = layers.Input(shape=[1])
    u = layers.Embedding(n_users + 1, features)(input_u)
    u = layers.Flatten()(u)
    u = layers.normalization.BatchNormalization()(u)
    
    nn = layers.concatenate([i, u])
    
    nn = layers.Dense(512, activation='relu')(nn)
    nn = layers.normalization.BatchNormalization()(nn)
#     nn = layers.Dropout(0.5)(nn)
    nn = layers.Dense(512, activation='relu')(nn)
    
    output =  layers.Dense(5, activation='softmax')(nn)
    
    model = models.Model([input_i, input_u], output)
    model.compile(optimizer='adamax', loss='categorical_crossentropy')
    return model

In [33]:
def deep_net():
    features = 48
    
    input_i = layers.Input(shape=[1])
    i = layers.Embedding(n_items + 1, features)(input_i)
    i = layers.Flatten()(i)
    i = layers.normalization.BatchNormalization()(i)

    input_u = layers.Input(shape=[1])
    u = layers.Embedding(n_users + 1, features)(input_u)
    u = layers.Flatten()(u)
    u = layers.normalization.BatchNormalization()(u)
    
    nn = layers.concatenate([i, u])
    
    nn = layers.Dense(1024, activation='relu')(nn)
    nn = layers.Dropout(0.5)(nn)
    nn = layers.normalization.BatchNormalization()(nn)
    nn = layers.Dense(512, activation='relu')(nn)
    nn = layers.Dropout(0.5)(nn)
    nn = layers.normalization.BatchNormalization()(nn)
    nn = layers.Dense(256, activation='relu')(nn)
    nn = layers.Dropout(0.5)(nn)
    nn = layers.normalization.BatchNormalization()(nn)
    nn = layers.Dense(128, activation='relu')(nn)
    
    output = layers.Dense(5, activation='softmax')(nn)
    
    model = models.Model([input_i, input_u], output)
    model.compile(optimizer='adamax', loss='categorical_crossentropy')
    
    return model

In [1]:
model = shallow_net()
model.summary()

if os.path.exists('NeuralNet_base.h5'):
    model = load_model('NeuralNet_base.h5')
else:
    history = model.fit([train_x.movie_id, train_x.user_id], y=categorical_train_y,  batch_size=20480, epochs=20, validation_data=([test_x.movie_id, test_x.user_id], categorical_test_y))
    model.save('NeuralNet_base.h5')
    plt.plot(history.history['val_loss'])
    plt.xlabel("Epochs")
    plt.ylabel("Test Error")

NameError: name 'shallow_net' is not defined

### 5.2 Model validation

In [39]:
model.evaluate([test_x.movie_id, test_x.user_id], categorical_test_y)

235391/235391 [==============================] - 11s 45us/step


1.3802265699990555

In [51]:
from sklearn.metrics import mean_squared_error

prediction_test = np.array([a[0] for a in model.predict([test_x.movie_id, test_x.user_id])])

prediction = (np.argmax(prediction_test,1)+1).tolist()

print (mean_squared_error(categorical_test_y, prediction))

ValueError: y_true and y_pred have different number of output (5!=1)

## 6. Building real test data

In [50]:
user_enc = LabelEncoder()
samples['user'] = user_enc.fit_transform(samples['user_id'].values)
n_users = samples['user'].nunique()

item_enc = LabelEncoder()
samples['movie'] = item_enc.fit_transform(samples['movie_id'].values)
n_movies = samples['movie'].nunique()

samples['rating'] = samples['rating'].values.astype(np.int)

X = samples[['user', 'movie']].values
X_array = [X[:,0], X[:,1]]

prediction = np.array([a[0] for a in model.predict(x=X_array)])

samples.drop('rating',axis = 1, inplace = True)
samples['rating'] = prediction

In [52]:
from helpersNeuralNet import create_csv

DATA_SUBMISSION = "data/submission_neuralnet_base.csv"
create_csv(DATA_SUBMISSION, samples)

AICrowd result : RMSE = 1.159 : Secondary = -0.056